# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Create a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

# Creating tables to run the following queries:

### 1. Display the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Display only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Display every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# The order of column names from .csv file:
# artist, firstName, gender, itemInsession, lastName, length, level, location, sessionID, song, userId

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history 
## that was heard during sessionId = 338 and itemInSession = 4
create_query1 = '''
CREATE TABLE IF NOT EXISTS music_history (
sessionId INT, 
itemInSession INT, 
artist TEXT, 
song TEXT, 
song_length DECIMAL, 
PRIMARY KEY(sessionId, itemInSession));
'''                   

try:
    session.execute(create_query1)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
## Assign the INSERT statements into the `query` variable
        query = '''
        INSERT INTO music_history (
        sessionId,
        itemInSession,
        artist,
        song,
        song_length)
        '''
        query = query + 'VALUES (%s, %s, %s, %s, %s);'
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Performing a SELECT to verify that the data have been inserted into each table

In [11]:
select_query1 = '''
SELECT artist, song, song_length
FROM music_history
WHERE sessionId = 338
AND itemInSession = 4;
'''

try:
    rows1 = session.execute(select_query1)
    lods1 = [] #lods == list of dicts
    for row in rows1:
        lods1.append({'artist':row.artist, 'song':row.song, 'song_length':row.song_length})
#         print(row.artist + ':', row.song + ' -', row.song_length, 'seconds')
except Exception as e:
    print(e)

In [12]:
pd.DataFrame(lods1)

,artist,song,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.3073


### REPEATING THE ABOVE THREE CELLS FOR EACH OF THE REMAINING QUESTIONS

In [13]:
## Query 2: Give me only the following: name of artist, 
## song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182

create_query2 = '''
CREATE TABLE IF NOT EXISTS user_history (
userId INT,
sessionId INT,
itemInSession INT,
artist TEXT, 
song TEXT,
firstName TEXT, 
lastName TEXT,
PRIMARY KEY(userId, sessionId, itemInSession));
'''                   

try:
    session.execute(create_query2)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = '''
        INSERT INTO user_history (
        userId,
        sessionId,
        itemInSession,
        artist,
        song,
        firstName,
        lastName)
        '''
        query = query + 'VALUES (%s, %s, %s, %s, %s, %s, %s);'
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [15]:
select_query2 = '''
SELECT artist, song, firstName, lastName
FROM user_history
WHERE userid = 10
AND sessionid = 182;
'''
    
try:
    rows2 = session.execute(select_query2)
    lods2 = [] #lods == list of dicts
    for row in rows2:
        lods2.append({'artist':row.artist, 'song':row.song, 'firstName':row.firstname, 'lastName':row.lastname})
except Exception as e:
    print(e)

In [16]:
pd.DataFrame(lods2)[['artist', 'song', 'firstName', 'lastName']]

,artist,song,firstName,lastName
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [17]:
## Query 3: Give me every user name (first and last) in my music app history 
## who listened to the song 'All Hands Against His Own'

create_query3 = '''
CREATE TABLE IF NOT EXISTS listener_history (
song TEXT,
userId INT, 
firstName TEXT, 
lastName TEXT,
PRIMARY KEY(song, userId));
'''                   

try:
    session.execute(create_query3)
except Exception as e:
    print(e)

In [18]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = '''
        INSERT INTO listener_history (
        song,
        userId,
        firstName,
        lastName)
        '''
        query = query + 'VALUES (%s, %s, %s, %s);'
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [19]:
select_query3 = '''
SELECT firstName, lastName
FROM listener_history
WHERE song = 'All Hands Against His Own';
'''

try:
    rows3 = session.execute(select_query3)
    lods3 = [] #lods == list of dicts
    for row in rows3:
        lods3.append({'firstName':row.firstname, 'lastName':row.lastname})
except Exception as e:
    print(e)

In [20]:
pd.DataFrame(lods3)

,firstName,lastName
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Droping the tables before closing out the sessions

In [21]:
## Drop the tables
drop_query1 = 'DROP TABLE music_history;' 
drop_query2 = 'DROP TABLE user_history;' 
drop_query3 = 'DROP TABLE listener_history;' 
drop_list = [drop_query1, drop_query2, drop_query3]

for q in drop_list:
    try:
        session.execute(q)
    except Exception as e:
        print(e)

### Closing the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()